In [2]:
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
import math

In [55]:
def make_normal_albedo(radius, albedo_value):
    len = 2 * radius + 1
    cen = radius
    normal = np.zeros((len, len, 3))
    albedo = np.full((len, len), albedo_value)
    mask = np.empty((len, len), dtype=bool)

    for i in range(len):
        for j in range(len):
            mask[i, j] = ((i-cen)**2 + (j-cen)**2 <= radius**2)
            kx = (j-cen) / radius
            ky = (cen-i) / radius
            kz = sqrt(max(1 - kx**2 - ky**2, 0))
            normal[i, j, 0] = kx
            normal[i, j, 1] = ky
            normal[i, j, 2] = kz

    albedo[~mask] = 0
    normal[~mask] = 0

    return (albedo, normal, mask)

In [56]:
def measurement(light, albedo, normal):
    B = (albedo * normal.T).T
    measurement = B @ light
    index = measurement < 0
    measurement[index] = 0

    return measurement

In [57]:
Light = []
filename = './src/'

albedo, normal, mask = make_normal_albedo(200, 1)
np.save('./gt_normal.npy', normal)
plt.imsave('./mask.png', albedo, cmap='gray')
phi = math.pi / 2.1

for i in range(100):
    theta = 2*math.pi / 100 * i
    light = [math.cos(theta)*math.cos(phi), math.sin(theta)*math.cos(phi), math.sin(phi)]
    M = measurement(light, albedo, normal)
    Light.append(light)
    np.save(filename+'img{}.npy'.format('%03d'%i), M)

Light = np.array(Light)
np.save('./lights.npy', Light)